In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
import random
import os

def sigmoid_norm(z):
    zn = z/np.std(z)
    zn = 1/(1 + np.exp(-zn))
    zn = (zn-0.5)*2.0
    return zn

#pdtemp = pd.read_csv('../tests/geneid2hugo.tsv',sep='\t')
pdtemp = pd.read_csv('tests/biomart_hugogeneid.tsv',sep='\t')
hugo2geneid = {row['HUGO']:row['GeneID'] for i,row in pdtemp.iterrows()}
geneid2hugo = {row['GeneID']:row['HUGO'] for i,row in pdtemp.iterrows()}

plt.rcParams['figure.dpi'] = 300
hist_colormap = 'Blues'

In [ ]:
# score_type can be 'ORF' or 'CRISPR'
score_type = 'ORF'
#score_type = 'CRISPR'

dfscores = pd.read_csv('data/'+score_type.lower()+'_scores_merged.tsv',sep='\t')
#dfscores = pd.read_csv('/mnt/c/Datas/MorphMap/crispr_scores_merged.tsv',sep='\t')
#label = 'CRISPR'
label = score_type
display(dfscores)


In [ ]:
# Create a randomized (reshuffled) version of dfscores

listofgenes = list(set(list(dfscores['GENE1']) + list(dfscores['GENE2'])))
print(len(listofgenes))

listofgenes_reshuffled = listofgenes.copy()
random.shuffle(listofgenes_reshuffled)

scorevalues_reshuffled = list(dfscores['ABS_'+label+'_SIM'])
random.shuffle(scorevalues_reshuffled)

gene1 = list(dfscores['GENE1'])
gene2 = list(dfscores['GENE2'])

pair_keys = {gene1[i]+'#'+gene2[i]:i for i,g in tqdm(enumerate(gene1))}
scores = list(dfscores['ABS_'+label+'_SIM'])
reshuffle_map = {l:listofgenes_reshuffled[i] for i,l in enumerate(listofgenes)}

scorevalues_reshuffled = []
for i,g in tqdm(enumerate(gene1)):
    key = reshuffle_map[gene1[i]]+'#'+reshuffle_map[gene2[i]]
    if not key in pair_keys:
        key = reshuffle_map[gene2[i]]+'#'+reshuffle_map[gene1[i]]
    scorevalues_reshuffled.append(scores[pair_keys[key]])
    

dfscores_reshuffled = dfscores.copy()
dfscores_reshuffled['ABS_'+label+'_SIM'] = scorevalues_reshuffled



In [ ]:
# show histogram in an interval
scores = dfscores[(dfscores['ABS_'+label+'_SIM']>=0.4)&(dfscores['ABS_'+label+'_SIM']<=0.41)]
sns.histplot(scores,x='gene_mf',bins=50,stat='probability')

In [ ]:
# unconditional density plots
fields = ['gene_mf','gene_bp','gene_pathway','unsupervised_max','weakly_supervised','strongly_supervised']
for field in fields:
    #sns.scatterplot(dfscores,x='ORF_SIM',y=field,s=1,c='k')
    sns.histplot(dfscores,x='ABS_'+label+'_SIM',y=field,cmap=hist_colormap,bins=(100,100),alpha=1.0,legend=True,stat='density',cbar=True)
    plt.show()



In [ ]:
# conditional density, both negative and positive values

fields = ['gene_mf','gene_bp','gene_pathway','unsupervised_max','weakly_supervised','strongly_supervised']
stepsize = 0.015
for field in fields:
    #sns.scatterplot(dfscores,x='ORF_SIM',y=field,s=1,c='k')
    orf_intervals = np.arange(-1,1,stepsize)
    for i,x in enumerate(orf_intervals):
        dfscores_int = dfscores.loc[(dfscores[label+'_SIM']>=x-stepsize/10)&(dfscores[label+'_SIM']<=x+stepsize*1.1)]
        if len(dfscores_int)>20:
            try:
                sns.histplot(dfscores_int,x=label+'_SIM',y=field,cmap=hist_colormap,bins=(1,50),alpha=1.0)
            except:
                k = 1
        
    plt.legend()
    plt.show()

In [ ]:
# conditional density, absolute values

fields = ['gene_mf','gene_bp','gene_pathway','unsupervised_max','weakly_supervised','strongly_supervised']
stepsize = 0.015
for field in fields:
    #sns.scatterplot(dfscores,x='ORF_SIM',y=field,s=1,c='k')
    orf_intervals = np.arange(-1,1,stepsize)
    for i,x in enumerate(orf_intervals):
        dfscores_int = dfscores.loc[(dfscores['ABS_'+label+'_SIM']>=x-stepsize/10)&(dfscores['ABS_'+label+'_SIM']<=x+stepsize*1.1)]
        if len(dfscores_int)>10:
            try:
                sns.histplot(dfscores_int,x='ABS_'+label+'_SIM',y=field,cmap=hist_colormap,bins=(1,50),alpha=1.0,kde=True)
            except:
                k = 1
        
    plt.legend()
    plt.show()

In [ ]:
# Plot explained fraction of gene pairs as a function of MorphMap similarity

colors  = ['r','g','b','m','y']

fields = ['gene_mf','gene_bp','gene_pathway','unsupervised_max','weakly_supervised','strongly_supervised']
fields = ['gene_mf','gene_pathway','gene_bp']
#fields = ['gene_mf','gene_pathway','gene_bp','unsupervised_max']
#fields = ['weakly_supervised','strongly_supervised']
normalize_explained_fraction = False
if score_type=='ORF':
    func_high_thresh = 0.7
else:
    func_high_thresh = 0.6
stepsize = 0.05
for k,field in enumerate(fields):
    orf_scores = np.array(dfscores[label+'_SIM'])
    #
    # the next line can be commented in order to see both positive and negative side of the distribution
    #
    orf_scores = np.array(dfscores['ABS_'+label+'_SIM'])
    func_scores = np.array(dfscores[field])
    expected__high_fraction = np.sum(func_scores>func_high_thresh)/func_scores.shape[0]
    print('Expected high fraction=',expected__high_fraction)
    orf_intervals = np.arange(-0.6,1,stepsize)
    observed_high_ratios = []
    for i,x in enumerate(orf_intervals):
        irx = np.where((orf_scores>x)&(orf_scores<x+stepsize))[0]
        sample = func_scores[irx]
        observed_high_fraction = np.sum(sample>func_high_thresh)/sample.shape[0]
        if normalize_explained_fraction:
            ratio = (observed_high_fraction-expected__high_fraction)/expected__high_fraction
        else:
            ratio = observed_high_fraction
        observed_high_ratios.append(ratio*100)
    plt.plot(orf_intervals,observed_high_ratios,colors[k]+'-',linewidth=5,label=field)
    plt.xlabel(label+' similarity')
    if normalize_explained_fraction:
        plt.ylabel('Enrichment relative difference, %')
    else:
        plt.ylabel('Explained fraction, %')
if normalize_explained_fraction:
    plt.title('Explained normalized fraction')
else:
    plt.title('Explained fraction of gene-gene links')

# Adding reshuffled baseline
    
for k,field in enumerate(fields):
    orf_scores = np.array(dfscores[label+'_SIM'])
    #
    # the next line can be commented in order to see both positive and negative side of the distribution
    #
    orf_scores = np.array(dfscores_reshuffled['ABS_'+label+'_SIM'])
    func_scores = np.array(dfscores_reshuffled[field])
    expected__high_fraction = np.sum(func_scores>func_high_thresh)/func_scores.shape[0]
    print('Expected high fraction=',expected__high_fraction)
    orf_intervals = np.arange(-0.6,1,stepsize)
    observed_high_ratios = []
    orf_intervals = np.arange(0.0,1.0,stepsize)
    for i,x in enumerate(orf_intervals):
        irx = np.where((orf_scores>x)&(orf_scores<x+stepsize))[0]
        sample = func_scores[irx]
        if len(irx)>15:
            observed_high_fraction = np.sum(sample>func_high_thresh)/sample.shape[0]
            if normalize_explained_fraction:
                ratio = (observed_high_fraction-expected__high_fraction)/expected__high_fraction
            else:
                ratio = observed_high_fraction
            observed_high_ratios.append(ratio*100)
        else:
            observed_high_ratios.append(ratio*100)
    plt.plot(orf_intervals,observed_high_ratios,colors[k]+'--',linewidth=3,label=field+'_reshuffled')

plt.legend()
plt.show()

In [ ]:
# Plot unexplained fraction of gene pairs as a function of MorphMap similarity

fields = ['gene_mf','gene_bp','gene_pathway','unsupervised_max','weakly_supervised','strongly_supervised']
fields = ['gene_mf','gene_bp','gene_pathway','unsupervised_max']

normalize_unexplained_fraction = False

for field in fields:
    orf_scores = np.array(dfscores['ORF_SIM'])
    func_scores = np.array(dfscores[field])
    func_low_thresh = 0.5
    expected__low_fraction = np.sum(func_scores<func_low_thresh)/func_scores.shape[0]
    print('Expected low fraction=',expected__low_fraction)
    stepsize = 0.05
    orf_intervals = np.arange(0,1,stepsize)
    observed_low_ratios = []
    #print(orf_intervals)
    for i,x in enumerate(orf_intervals):
        irx = np.where((orf_scores>x)&(orf_scores<x+stepsize))[0]
        #print(irx)
        sample = func_scores[irx]
        observed_low_fraction = np.sum(sample<func_low_thresh)/sample.shape[0]
        if normalize_unexplained_fraction:
            ratio = (observed_low_fraction-expected__low_fraction)/expected__low_fraction
        else:
            ratio = observed_low_fraction
        observed_low_ratios.append(ratio*100)
        #print('observed_high_fraction=',observed_high_fraction)
        #dfscores['INT'] = np.array(((orf_scores>x)&(orf_scores<x+stepsize)))
        #sns.displot(dfscores,x=field,hue='INT',kind='kde',common_norm=False)
        #plt.title('[{:2.2f},{:2.2f}]'.format(x,x+stepsize))
    plt.plot(orf_intervals,observed_low_ratios,'-',linewidth=5,label=field)
    plt.xlabel('ORF similarity')
    plt.ylabel('Enrichment relative difference, %')
plt.legend()
if normalize_unexplained_fraction:
    plt.title('Unexplained normalized fraction')
else:
    plt.title('Unexplained fraction of gene-gene links')
plt.show()

In [ ]:
#sns.histplot(dfscores_old,x='gene_pathway',y='gene_bp',bins=(100,100),cmap=hist_colormap)

In [ ]:
# This cell is for comparing two functional embedding versions

if os.path.exists('data/'+score_type.lower()+'_scores_merged.old'):
    dfscores_old = pd.read_csv('data/'+score_type.lower()+'_scores_merged.old',sep='\t')
    gene1_old = list(dfscores_old['GENE1'])
    gene2_old = list(dfscores_old['GENE2'])
    pair_keys_old = {gene1_old[i]+'#'+gene2_old[i]:i for i,g in tqdm(enumerate(gene1_old))}
    mf_old = dfscores_old['gene_mf']
    bp_old = dfscores_old['gene_bp']
    pt_old = dfscores_old['gene_pathway']

    dfscores = pd.read_csv('data/'+score_type.lower()+'_scores_merged.tsv',sep='\t')
    gene1 = list(dfscores['GENE1'])
    gene2 = list(dfscores['GENE2'])
    pair_keys = {gene1[i]+'#'+gene2[i]:i for i,g in tqdm(enumerate(gene1))}

    mf_old_reordered = []
    bp_old_reordered = []
    pt_old_reordered = []

    for i,g in tqdm(enumerate(gene1)):
        key = gene1[i]+'#'+gene2[i]
        if key in pair_keys_old:
            mf_old_reordered.append(mf_old[pair_keys_old[key]])
        else:
            mf_old_reordered.append(np.nan)
    bp_old_reordered = []
    for i,g in tqdm(enumerate(gene1)):
        key = gene1[i]+'#'+gene2[i]
        if key in pair_keys_old:
            bp_old_reordered.append(bp_old[pair_keys_old[key]])
        else:
            bp_old_reordered.append(np.nan)
    pt_old_reordered = []
    for i,g in tqdm(enumerate(gene1)):
        key = gene1[i]+'#'+gene2[i]
        if key in pair_keys_old:
            pt_old_reordered.append(pt_old[pair_keys_old[key]])
        else:
            pt_old_reordered.append(np.nan)

    dfscores['gene_mf.old'] = mf_old_reordered
    dfscores['gene_bp.old'] = bp_old_reordered
    dfscores['gene_pathway.old'] = pt_old_reordered

In [ ]:
# This cell is for comparing two functional embedding versions
if os.path.exists('data/'+score_type.lower()+'_scores_merged.old'):
    sns.histplot(dfscores,x='gene_mf',y='gene_mf.old',bins=[100,100],cmap=hist_colormap)
    plt.show()
    sns.histplot(dfscores,x='gene_bp',y='gene_bp.old',bins=[100,100],cmap=hist_colormap)
    plt.show()
    sns.histplot(dfscores,x='gene_pathway',y='gene_pathway.old',bins=[100,100],cmap=hist_colormap)
    plt.show()

In [ ]:
display(dfscores[dfscores['gene_bp']>0.95])